In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "geodatasets",
#     "geopandas[all]",
#     "microjson",
#     "zarr",
#     "shapely",
# ]
# ///

In [62]:
import json

import geodatasets
import geopandas as gpd
import shapely

# Intro to shape formats


## In-Memory Representation: geopandas

Let's start by looking at how the in-memory shapes are stored by lookinga at a sample dataset. We see that we have a collection of different geometries along with a custom in-memory pandas data type, `geometry`

In [32]:
colombia = gpd.read_file(geodatasets.get_path("geoda.malaria"))

In [41]:
colombia["geometry"]

0       POLYGON ((-71.32639 11.84789, -71.33579 11.855...
1       POLYGON ((-72.42191 11.79824, -72.4198 11.795,...
2       POLYGON ((-72.1891 11.5242, -72.1833 11.5323, ...
3       POLYGON ((-72.638 11.3679, -72.6259 11.3499, -...
4       POLYGON ((-74.77489 10.93158, -74.7753 10.9338...
                              ...                        
1063    POLYGON ((-77.1137 0.417, -77.0659 0.4878, -77...
1064    MULTIPOLYGON (((-75.33747 10.69339, -75.3129 1...
1065    MULTIPOLYGON (((-78.2675 12.0632, -78.2594 12....
1066    MULTIPOLYGON (((-75.93092 9.42257, -75.93809 9...
1067    MULTIPOLYGON (((-77.2433 12.3945, -77.2439 12....
Name: geometry, Length: 1068, dtype: geometry

`shapely` offers a way to see the "true memory layout" i.e., from the [`shapely` docs](https://shapely.readthedocs.io/en/latest/reference/shapely.to_ragged_array.html):

"This follows the in-memory layout of the variable size list arrays defined by Apache Arrow, as specified for geometries by the GeoArrow project: https://github.com/geoarrow/geoarrow"

In [37]:
shapely.to_ragged_array(colombia["geometry"])

(<GeometryType.MULTIPOLYGON: 6>,
 array([[-71.3263855 ,  11.84788799],
        [-71.33579254,  11.8552742 ],
        [-71.34572601,  11.8481369 ],
        ...,
        [-77.20980072,  12.28409958],
        [-77.20120239,  12.28569984],
        [-77.19080353,  12.2840004 ]], shape=(87955, 2)),
 (array([    0,   303,   401, ..., 87790, 87842, 87955], shape=(1119,)),
  array([   0,    1,    2, ..., 1116, 1117, 1118], shape=(1119,)),
  array([   0,    1,    2, ..., 1114, 1116, 1118], shape=(1069,))))

First we have the geometry type, which is `MULTIPOLYGON` because our underlying types are mixed (both `POLYGON` and `MULTIPOLYGON`).

In [42]:
shapely.to_ragged_array(colombia["geometry"])[0]

<GeometryType.MULTIPOLYGON: 6>

Next we have the underlying data, made up of xy coordinates

In [39]:
shapely.to_ragged_array(colombia["geometry"])[1]

array([[-71.3263855 ,  11.84788799],
       [-71.33579254,  11.8552742 ],
       [-71.34572601,  11.8481369 ],
       ...,
       [-77.20980072,  12.28409958],
       [-77.20120239,  12.28569984],
       [-77.19080353,  12.2840004 ]], shape=(87955, 2))

Lastly, the offsets - the first array has the offsets of each geometry's data, the last two contiain inner and outer rings if applicable.

In [40]:
shapely.to_ragged_array(colombia["geometry"])[2]

(array([    0,   303,   401, ..., 87790, 87842, 87955], shape=(1119,)),
 array([   0,    1,    2, ..., 1116, 1117, 1118], shape=(1119,)),
 array([   0,    1,    2, ..., 1114, 1116, 1118], shape=(1069,)))

## Working with geojson and geopandas

Here is a short code snippet showing how to roundtrip the data properly. The `get_parts` call is not necessary but roundtrips the data back to the representation you see instead of a likely-less-helpful `GEOMETRYCOLLECTION` (see https://shapely.readthedocs.io/en/latest/reference/shapely.from_geojson.html#shapely.from_geojson)

In [79]:
shapely.get_parts(
    shapely.from_geojson(
        json.dumps(
            {
                "type": "FeatureCollection",
                "features": colombia["geometry"]
                .map(
                    lambda x: {
                        "type": "Feature",
                        "geometry": json.loads(shapely.to_geojson(x)),
                        "properties": None,
                    }
                )
                .to_list(),
            }
        )
    )
)

array([<POLYGON ((-71.326 11.848, -71.336 11.855, -71.346 11.848, -71.353 11.847, -...>,
       <POLYGON ((-72.422 11.798, -72.42 11.795, -72.407 11.774, -72.386 11.745, -7...>,
       <POLYGON ((-72.189 11.524, -72.183 11.532, -72.17 11.549, -72.165 11.565, -7...>,
       ...,
       <MULTIPOLYGON (((-78.268 12.063, -78.259 12.046, -78.259 12.041, -78.257 12....>,
       <MULTIPOLYGON (((-75.931 9.423, -75.938 9.429, -75.938 9.431, -75.935 9.433,...>,
       <MULTIPOLYGON (((-77.243 12.394, -77.244 12.384, -77.25 12.37, -77.259 12.36...>],
      shape=(1068,), dtype=object)

## Working with {Micro,Tile}JSON

For a more detailed intro see https://polusai.github.io/microjson/tiling_demo/ - this is mostly copy-and-paste from there.  For now we're ignoring most of the features around transformations and such.

There is also information in that notebook on how to generate more data.

In [97]:
import json
import os
from pathlib import Path

from microjson.tilemodel import TileJSON, TileLayer, TileModel
from microjson.tilewriter import TileWriter, extract_fields_ranges_enums, getbounds

In [104]:
microjson_data_path = "test.json"
with open(microjson_data_path, "w") as f:
    f.write(
        json.dumps(
            {
                "type": "FeatureCollection",
                "features": colombia["geometry"]
                .map(
                    lambda x: {
                        "type": "Feature",
                        "geometry": json.loads(shapely.to_geojson(x)),
                        "properties": {},  # non-null
                    }
                )
                .to_list(),
            }
        )
    )

In [105]:
# Extract fields, ranges, and enums from the MicroJSON data
field_names, field_ranges, field_enums = extract_fields_ranges_enums(
    microjson_data_path
)

print("Extracted field names:")
print(field_names)

print("\nExtracted field ranges:")
print(field_ranges)

print("\nExtracted field enums:")
print(field_enums)

Extracted field names:
{}

Extracted field ranges:
{}

Extracted field enums:
{}


In [106]:
vector_layers = [
    TileLayer(
        id="polygon-layer",
        fields=field_names,
        minzoom=0,
        maxzoom=10,
        description="Layer containing polygon data",
        fieldranges=field_ranges,
        fieldenums=field_enums,
    )
]

print("Vector layer defined with the following properties:")
print(f"ID: {vector_layers[0].id}")
print(f"Fields: {vector_layers[0].fields}")
print(f"Zoom range: {vector_layers[0].minzoom} - {vector_layers[0].maxzoom}")

Vector layer defined with the following properties:
ID: polygon-layer
Fields: {}
Zoom range: 0 - 10


In [107]:
# Get bounds of the data (square=True ensures the bounds form a square)
maxbounds = getbounds(microjson_data_path, square=True)
print(f"Bounds: {maxbounds}")

# Calculate the center of the bounds
center = [0, (maxbounds[0] + maxbounds[2]) / 2, (maxbounds[1] + maxbounds[3]) / 2]
print(f"Center: {center}")

Bounds: [-79.04722595214844, -4.2304840087890625, -62.35343360900879, 12.463308334350586]
Center: [0, -70.70032978057861, 4.116412162780762]


In [108]:
# Create output directory for tiles
os.makedirs("tiles", exist_ok=True)

# Instantiate TileModel with our settings
tile_model = TileModel(
    tilejson="3.0.0",
    tiles=[Path("tiles/{z}/{x}/{y}.json")],  # Local path or URL
    name="Example Tile Layer",
    description="A TileJSON example incorporating MicroJSON data",
    version="1.0.0",
    attribution="Polus AI",
    minzoom=0,
    maxzoom=7,
    bounds=maxbounds,
    center=center,
    vector_layers=vector_layers,
)

# Create the root model with our TileModel instance
tileobj = TileJSON(root=tile_model)

# Display the TileJSON specification
print("TileJSON specification:")
print(tileobj.model_dump_json(indent=2))

TileJSON specification:
{
  "tilejson": "3.0.0",
  "tiles": [
    "tiles/{z}/{x}/{y}.json"
  ],
  "name": "Example Tile Layer",
  "description": "A TileJSON example incorporating MicroJSON data",
  "version": "1.0.0",
  "attribution": "Polus AI",
  "template": null,
  "legend": null,
  "scheme": null,
  "grids": null,
  "data": null,
  "minzoom": 0,
  "maxzoom": 7,
  "bounds": [
    -79.04722595214844,
    -4.2304840087890625,
    -62.35343360900879,
    12.463308334350586
  ],
  "center": [
    0.0,
    -70.70032978057861,
    4.116412162780762
  ],
  "fillzoom": null,
  "vector_layers": [
    {
      "id": "polygon-layer",
      "fields": {},
      "minzoom": 0,
      "maxzoom": 10,
      "description": "Layer containing polygon data",
      "fieldranges": {},
      "fieldenums": {},
      "fielddescriptions": null
    }
  ],
  "multiscale": null,
  "scale_factor": null
}


In [109]:
# Export to tilejson
with open("tiles/metadata.json", "w") as f:
    f.write(tileobj.model_dump_json(indent=2))

print("TileJSON metadata exported to tiles/metadata.json")

TileJSON metadata exported to tiles/metadata.json


In [110]:
# Initialize the TileWriter
handler = TileWriter(tile_model, pbf=False)

# Convert MicroJSON to tiles
handler.microjson2tiles(microjson_data_path, validate=False)

print("Vector tiles generated successfully!")

# List the generated tile directories to verify
tile_dirs = [d for d in os.listdir("tiles") if os.path.isdir(os.path.join("tiles", d))]
print(f"Generated tile zoom levels: {tile_dirs}")

Vector tiles generated successfully!
Generated tile zoom levels: ['0', '7', '6', '1', '4', '3', '2', '5']
